In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/local_nikhil/.conda/envs/position_lens/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading Model

In [2]:
if "model" in locals():
    del model
    torch.cuda.empty_cache()
if "tokenizer" in locals():
    del tokenizer

path = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(path)
model = model.half()
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(path, padding_side="left")

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]


# Loading Data

In [3]:
with open("data/unexpected_contents.jsonl") as f:
    unexpected_contents = [json.loads(line) for line in f]

with open("data/unexpected_transfer.jsonl") as f:
    unexpected_transfer = [json.loads(line) for line in f]

In [4]:
tasks = []
for data in unexpected_contents + unexpected_transfer:
    for i in range(3):
        inp = {
            "input": data['prompts'][i],
            "target": data[f'target_{i+1}']
        }
        tasks.append(inp)

In [5]:
# Create a dataset from tasks list
dataset = Dataset.from_list(tasks).with_format("torch")

# Create a collate function to tokenize and pad the data
def collate_fn(examples):
    inputs = tokenizer(
        [ex["input"] for ex in examples],
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    if (
        model.config.architectures[0] == "GPT2LMHeadModel"
        or model.config.architectures[0] == "GPTNeoXForCausalLM"
        or model.config.architectures[0] == "GPTJForCausalLM"
    ):
        inputs["target"] = [tokenizer.encode(" " + ex["target"])[0] for ex in examples]
    elif model.config.architectures[0] == "LlamaForCausalLM" or model.config.architectures[0] == "LlaMAForCausalLM":
        inputs["target"] = [tokenizer.encode(ex["target"])[1] for ex in examples]
    elif model.config.architectures[0] == "MistralForCausalLM":
        inputs["target"] = [tokenizer.encode(" " + ex["target"])[2] for ex in examples]
    return inputs

# Create a DataLoader
dataloader = DataLoader(
    dataset, collate_fn=collate_fn, batch_size=1, shuffle=False
)

# Model Performance

In [7]:
correct, total = 0, 0
with torch.no_grad():
    for inp in tqdm(dataloader, total=len(dataloader)):
        inp["input_ids"] = inp["input_ids"].to(device)
        inp["attention_mask"] = inp["attention_mask"].to(device)

        outputs = model(inp["input_ids"])
        logits = outputs.logits
        predicted_ids = torch.argmax(logits[:, -1], dim=-1)

        for i, predicted_id in enumerate(predicted_ids):
            if predicted_id == inp["target"][i]:
                # print(f"{tokenizer.decode(predicted_id)} == {tokenizer.decode(inp['target'][i])}")
                correct += 1
            else:
                pass
                # print(f"{tokenizer.decode(predicted_id)} != {tokenizer.decode(inp['target'][i])}")
            total += 1

print(f"Accuracy: {correct/total:.2f}")

  1%|          | 4/549 [00:00<00:26, 20.95it/s]

beer == beer
wine == wine
beer != wine
audio == audio
video == video
audio != video


  2%|▏         | 11/549 [00:00<00:19, 27.24it/s]

ice == ice
jam == jam
jam == jam
salt == salt
sugar == sugar
sugar == sugar
pens == pens


  3%|▎         | 19/549 [00:00<00:18, 28.99it/s]

...

Oxford != Not
Not == Not
Finance != Internal
Finance != Internal
Finance != Internal


 99%|█████████▉| 545/549 [00:19<00:00, 24.02it/s]

templates != templates
docs != templates
templates != templates
X != Y
X != Y


100%|██████████| 549/549 [00:19<00:00, 27.48it/s]

X != Y
box == box
basket != box
basket != box
Accuracy: 0.53
